In [2]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
from huggingface_hub import login, InferenceClient
from transformers import AutoTokenizer

In [3]:
# Load environment variables in a file called .env

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
hf_token = os.environ['HF_TOKEN']

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

if hf_token:
    print(f"Huggingface Key exists and begins {hf_token[:8]}")
else:
    print("Huggingface Key not set")
    
# Set path for g++ compiler
new_path = r"D:\w64devkit\bin"  # Windows
# new_path = "/usr/local/bin"  # Linux/Mac
os.environ['PATH'] = new_path + os.pathsep + os.environ['PATH']

# Set frontier models
openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

# Set open source models
code_qwen = "Qwen/CodeQwen1.5-7B-Chat"
CODE_QWEN_URL = "https://h1vdol7jxhje3mpn.us-east-1.aws.endpoints.huggingface.cloud"

#login to huggingface

login(hf_token, add_to_git_credential=True)

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyC0
Huggingface Key exists and begins hf_EVcWN


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
def system_prompt_for_converting(from_code, to_code):
    system_prompt = f"You are an assistant that reimplements {from_code} code in high performance {to_code}. "
    system_prompt += f"Respond only with {to_code} code; use comments sparingly and do not provide any explanation other than occasional comments. "
    system_prompt += f"The {to_code} response needs to produce an identical output in the fastest possible time."
    return system_prompt

In [5]:
def user_prompt_for_converting(code_to_convert, from_code, to_code):
    user_prompt = f"Rewrite this {from_code} code in {to_code} with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += f"Respond only with {to_code} code; do not explain your work other than a few comments. "
    
    if to_code == "C++":
        user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
        
    user_prompt += code_to_convert
    
    return user_prompt

In [6]:
def messages_for_converting(code_to_convert, from_code, to_code):
    return [
        {"role": "system", "content": system_prompt_for_converting(from_code, to_code)},
        {"role": "user", "content": user_prompt_for_converting(code_to_convert, from_code, to_code)}
    ]

In [7]:
def write_output_cpp(generated_code):
    code = generated_code.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [20]:
def stream_gpt(messages):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages, stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [21]:
def stream_claude(user_message, system_message):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_message}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [22]:
def stream_code_qwen(messages):
    tokenizer = AutoTokenizer.from_pretrained(code_qwen)
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    client = InferenceClient(CODE_QWEN_URL, token=hf_token)
    stream = client.text_generation(text, stream=True, details=True, max_new_tokens=3000)
    result = ""
    for r in stream:
        result += r.token.text
        yield result   

In [23]:
def execute_cpp(code):
        write_output_cpp(code)
        try:
            compile_cmd = ["g++", "-std=c++17", "-o", "optimized", "optimized.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["optimized.exe"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [24]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [25]:
def execute_code(code,program_language):
    if program_language=="C++":
        return execute_cpp(code)
    elif program_language=="Python":
         return execute_python(code)
    else:
        raise ValueError(f"Unknown program language {program_language}")

In [26]:
def convert_code(code_to_convert, model, from_code, to_code):
    if model=="GPT":
        result = stream_gpt(messages_for_converting(code_to_convert, from_code, to_code))
    elif model=="Claude":
        result = stream_claude(user_prompt_for_converting(code_to_convert, from_code, to_code), system_prompt_for_converting(from_code, to_code))
    elif model=="CodeQwen":
        result = stream_code_qwen(messages_for_converting(code_to_convert, from_code, to_code))
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far    

In [15]:
css = """
.code {background-color: #306998;}
.output {background-color: #050;}
"""

In [27]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        from_code = gr.Dropdown(["C++", "Python"], label="Select code base", value="Python")
        code_to_convert = gr.Textbox(label="Code base:", value=python_simple, lines=10)
        to_code = gr.Dropdown(["C++", "Python"], label="Select convert to", value="C++")
        generated_code = gr.Textbox(label=f"Generated code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude", "CodeQwen"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        code_to_convert_out = gr.TextArea(label="Python result:", elem_classes=["code"])
        generated_code_out = gr.TextArea(label="C++ result:", elem_classes=["output"])

    convert.click(convert_code, inputs=[code_to_convert, model, from_code, to_code], outputs=[generated_code])
    python_run.click(execute_code, inputs=[code_to_convert, from_code], outputs=[code_to_convert_out])
    cpp_run.click(execute_code, inputs=[generated_code, to_code], outputs=[generated_code_out])

ui.launch(inbrowser=True)

Traceback (most recent call last):
  File "D:\Anaconda\envs\llms\Lib\site-packages\gradio\queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Anaconda\envs\llms\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Anaconda\envs\llms\Lib\site-packages\gradio\blocks.py", line 2220, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Anaconda\envs\llms\Lib\site-packages\gradio\blocks.py", line 1731, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Anaconda\envs\llms\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

In [17]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [18]:
python_simple = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""